# H３０地価公示csvデータに緯度・経度情報を付与する方法

## Pandasをインポート

In [1]:
import pandas as pd

## H３０地価公示データ（事前配布）を読み込む
エンコーディングに注意

In [2]:
koji_30 = pd.read_csv("kouji30.csv",header=0,encoding='Shift_JISx0213')

## H３０地価公示データの概要を確認

In [3]:
koji_30.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25988 entries, 0 to 25987
Data columns (total 56 columns):
標準地番号（都道府県市区町村コード）    25988 non-null int64
標準地番号（用途）             25988 non-null int64
標準地番号（連番）             25988 non-null int64
全国主要地点                25988 non-null int64
連番（県単位）               25988 non-null int64
都道府県名                 25988 non-null object
市郡名                   25988 non-null object
区町村名                  25988 non-null object
地名                    25988 non-null object
住居表示                  25988 non-null object
所在並びに地番               25988 non-null object
当年価格                  25988 non-null int64
前年価格                  25988 non-null object
対前年変動率（％）             25988 non-null object
地積                    25988 non-null int64
地積・内私道分               25988 non-null int64
形状区分                  25988 non-null int64
（比率）間口                25988 non-null int64
（比率）奥行                25988 non-null int64
利用区分                  25988 non-null int64
構造                    2

## H２９地価公示データ（国土数値情報）を読み込む
エンコーディングに注意

In [4]:
koji_29 = pd.read_csv("L01-29P-2K.csv",header=0,encoding='Shift_JISx0213')

## H２９地価公示データの概要を確認

In [5]:
koji_29.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25988 entries, 0 to 25987
Columns: 107 entries, 経度 to 属性移動Ｈ２９
dtypes: bool(1), float64(2), int64(88), object(16)
memory usage: 21.0+ MB


## H３０地価公示データの列の余分な空白を削除

In [6]:
koji_30["都道府県名_1"] = koji_30.都道府県名.str.strip()
koji_30["市郡名_1"] = koji_30.市郡名.str.strip()
koji_30["区町村名_1"] = koji_30.区町村名.str.strip()
koji_30["住居表示_1"] = koji_30.住居表示.str.strip()
koji_30["所在並びに地番_1"] = koji_30.所在並びに地番.str.strip()

## H３０地価公示データでkey_1を作成（住居表示）

In [7]:
koji_30["key_1"] = koji_30.都道府県名_1 + "　" + koji_30.市郡名_1 + koji_30.区町村名_1 + koji_30.住居表示_1

## H３０地価公示データでkey_1を作成（所在地並びに地番）

In [8]:
koji_30["key_2"] = koji_30.都道府県名_1 + "　" + koji_30.市郡名_1 + koji_30.区町村名_1 + koji_30.所在並びに地番_1

## H３０のkey_1とH２９の住居表示をキーにしてマージ（merged_1)

In [9]:
merged_1 = pd.merge(koji_30, koji_29, left_on = "key_1", right_on = "住居表示")

## merged_1の概要を表示

In [10]:
merged_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11292 entries, 0 to 11291
Columns: 170 entries, 標準地番号（都道府県市区町村コード） to 属性移動Ｈ２９
dtypes: bool(1), float64(3), int64(129), object(37)
memory usage: 14.7+ MB


## H３０のkey_2とH２９の住居表示をキーにしてマージ（merged_2)

In [11]:
merged_2 = pd.merge(koji_30, koji_29, left_on = "key_2", right_on = "住居表示")

## merged_2の概要を表示

In [12]:
merged_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14288 entries, 0 to 14287
Columns: 170 entries, 標準地番号（都道府県市区町村コード） to 属性移動Ｈ２９
dtypes: bool(1), float64(3), int64(129), object(37)
memory usage: 18.5+ MB


## merged_1とmerged_2を縦方向に連結(concat)

In [13]:
merged_3 = pd.concat([merged_1, merged_2])

## merged_3の概要を表示

In [14]:
merged_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25580 entries, 0 to 14287
Columns: 170 entries, 標準地番号（都道府県市区町村コード） to 属性移動Ｈ２９
dtypes: bool(1), float64(3), int64(129), object(37)
memory usage: 33.2+ MB


## 緯度経度データを秒から度（１０進法）に変換
/3600

In [15]:
merged_3["lon"] = merged_3.経度/3600
merged_3["lat"] = merged_3.緯度/3600

## merged_3の概要を表示

In [16]:
merged_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25580 entries, 0 to 14287
Columns: 172 entries, 標準地番号（都道府県市区町村コード） to lat
dtypes: bool(1), float64(5), int64(129), object(37)
memory usage: 33.6+ MB


## merged_3をcsvに吐き出す(merged_final.csv)

In [17]:
merged_3.to_csv("merged_final.csv", encoding='Shift_JISx0213')